In [1]:
!pip install planet

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for requests-futures: filename=requests_futures-0.9.7-cp36-none-any.whl size=5056 sha256=1b53c14928e41e5e9ef482ebfb33010bef62e2923ec24df4f005c12a744f1c38
  Stored in directory: /root/.cache/pip/wheels/71/fb/7a/0caacf78e523a182dadd1d042792801cfd53f27c77b945b64a
Successfully built requests-futures


In [2]:
import os
from shapely import geometry
import pickle
import json
import regex as re
import requests
import pandas as pd
from planet import api
import cv2
import csv
from requests.auth import HTTPBasicAuth
from planet.api import filters
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
def p(data):
    print(json.dumps(data, indent=2))

Mounted at /content/drive


In [ ]:
# API Key stored as an env variable
client = api.ClientV1()
client = api.ClientV1(api_key="YOUR_KEY_HERE")
print(client.auth.value)

In [ ]:
# Load in areas to clip with that were created in a GIS
with open("/content/drive/My Drive/CS_229_project/areas2.geojson") as f:
    esri_output  = json.loads(f.read())

In [7]:
#converts esri multipolygon geojson to planet geojson
all_areas = [item['geometry']['coordinates'] for item in esri_output['features']]
# grab the siteNum for batch downloads
siteNum = [item['properties']['Site_Number'] for item in esri_output['features']]
coords = {}
i = 0
for site in siteNum:
  coords[site] = all_areas[i]
  i += 1
coords.keys()

dict_keys([2, 16, 113, 1103, 1302, 1602, 1701, 2005, 4002, 5005, 7, 5001, 3, 8001, 8005, 5, 12, 27, 1004, 2002, 4001, 4003, 9004, 9010])

In [8]:
# Grab one study area
geom = { 
  "type": "MultiPolygon",
  "coordinates": all_areas
}

Single Query for detailed information

In [9]:
# get images that overlap with our AOI 
geo_filter = filters.geom_filter(geom)
# date filters
year = 2020
start_date = datetime(year=year, month=1, day=1)
end_date = datetime(year=year, month=9, day=30)
date_filter = filters.date_range('acquired', gte=start_date, lte = end_date)
# cloud filter
cloud_filter = filters.range_filter('cloud_cover', lte=0.3)
# only visual assets
asset_filter = {'config': ["visual"], 'type': 'AssetFilter'}
# combine our geo, date, cloud filters
and_filter = filters.and_filter(date_filter, cloud_filter, geo_filter, asset_filter)

In [10]:
# Get stats from a query
item_type = ["PSScene3Band"]
req = filters.build_search_request(and_filter, item_type, interval = 'year')
stats = client.stats(req).get()
n = stats['buckets'][0]['count']
p(stats)

{
  "buckets": [
    {
      "count": 7666,
      "start_time": "2020-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


In [11]:
item_type = ["PSScene3Band"]
# API request object
req = filters.build_search_request(and_filter, item_type)
res = client.quick_search(req)
data = []
coordsData = []
i = 0
for item in res.items_iter(n):
  i +=1
  for area in coords.keys():
    geom1 = {"type": "Polygon","coordinates": coords.get(area)}
    aoi = geometry.shape(geom1)
    g = geometry.shape(item["geometry"])
    # Optional filter for picture containing aoi
    if g.contains(aoi):
      #print(item["id"])
      context = [area, item['id'], item['properties']['strip_id'], item['properties']['cloud_cover']]
      coordsData.append([area, item['id'], item['geometry']['coordinates']])
      data.append(context)
i

7666

In [12]:
# Store the coordinates for viewing
coords_pd = pd.DataFrame(coordsData, columns= ["site", "file", 'coordinates'])

In [13]:
# Remove duplicate photos over multiple sites
data_pd = pd.DataFrame(data, columns= ["site", "file", 'strip_id', 'cloud_cover'])
data_pd_unique = data_pd.drop_duplicates(subset = ["file"]).sort_values("file")
data_pd_unique

,site,file,strip_id,cloud_cover
1232,4001,20200101_181109_1010,2991674,0.05
1228,1302,20200101_183712_29_1065,2992370,0.12
1231,16,20200102_181738_0f4e,2994320,0.05
1229,1103,20200102_183657_09_1058,2993849,0.10
1230,1302,20200102_183659_11_1058,2993849,0.12
...,...,...,...,...
3,2002,20200929_174652_39_222b,3768214,0.00
8,8005,20200929_174654_60_222b,3768214,0.00
6,1602,20200929_181638_103b,3769024,0.01
7,4002,20200929_181641_103b,3769024,0.01


In [ ]:
# Save the file for future use
with open("/content/drive/My Drive/CS_229_project/img_data/fileToSite/file2siteGroup2lte30" + str(year) + ".txt", "wb") as fp:
  pickle.dump(data_pd_unique,fp)